In [3]:
import rosbag
PATH="/home/xuhao/Dropbox/data/TRO2020-SwarmLocal/tro2020-bags/pre/"
PATHOUT="/home/xuhao/bags/swarm_local_2020_12/"
input_bag = PATH +"drone1-swarm_local_2020-12-13-21-31-51.bag"
output_bag = PATHOUT + "drone1-swarm_local_2020-12-13-21-31-51_processed.bag"
moveout_topics = {
    "/swarm_drones/node_detected",
    "/swarm_drones/swarm_drone_fused",
    "/swarm_drones/swarm_drone_fused_relative",
    "/swarm_drones/swarm_frame",
    "/swarm_drones/swarm_frame_predict"
}
#Moveout node_detected
with rosbag.Bag(output_bag, 'w') as outbag:
    for topic, msg, t in rosbag.Bag(input_bag).read_messages():
        if topic in moveout_topics:
            pass
        else:
            #outbag.write(topic, msg, msg.header.stamp if msg._has_header else t)
            outbag.write(topic, msg, t)            

In [17]:
import rosbag
import os
PATH="/home/xuhao/bags/swarm_local_2020_12/"
PATHOUT="/home/xuhao/bags/swarm_local_2020_12/"
bags = ["drone1-swarm_local_raw_2020-12-16-20-00-38.bag",
                "drone2-swarm_local_raw_2020-12-16-20-00-34.bag"]
#t_offset = 20
t_offset = 90
t0 = 0
for bag in bags:
    for topic, msg, t in rosbag.Bag(PATH+bag).read_messages():
        print(f"Bag {bag} start at {t.to_sec()}")
        if t.to_sec() > t0:
            t0 = t.to_sec()
            break
        else:
            break
            
print("Will use", t0, "as start")

ros_t0 = None
lps_t1 = None
moveout_topics = {
    "/uwb_node/incoming_broadcast_data"
}


for bag in bags:
    print(f"Converting {bag}")
    #cmd = f"/opt/ros/melodic/bin/rosbag filter {PATH}{bag} {PATHOUT}filtered-{bag} 't.to_sec() >= {t}'"
    #print(os.popen(cmd).read())
    output_bag = PATHOUT+"filtered-"+bag;
    with rosbag.Bag(output_bag, 'w') as outbag:
        for topic, msg, t in rosbag.Bag(PATH+bag).read_messages():
            if topic in moveout_topics:
                pass
            else:
                if t.to_sec() > t0 + t_offset:
                    outbag.write(topic, msg, t)

Bag drone1-swarm_local_raw_2020-12-16-20-00-38.bag start at 1608120038.617251
Bag drone2-swarm_local_raw_2020-12-16-20-00-34.bag start at 1608120034.5863779
Will use 1608120038.617251 as start
Converting drone1-swarm_local_raw_2020-12-16-20-00-38.bag
Converting drone2-swarm_local_raw_2020-12-16-20-00-34.bag


In [ ]:
ros_t0 = None
lps_t1 = None
for bag in bags:
    output_bag = PATHOUT+"filtered-"+bag;
    with rosbag.Bag(output_bag, 'w') as outbag:
        for topic, msg, t in rosbag.Bag(PATH+bag).read_messages():
            if t < t0:
                continue
            if bag ==  "drone2-swarm_local_raw_2020-12-12.bag":
                if topic == "/uwb_node/remote_nodes":
                    if ros_t0 is None:
                        ros_t0 = msg.header.stamp.to_sec()
                    if lps_t1 is None:
                        lps_t1 = msg.sys_time
                    lps_time = (msg.header.stamp.to_sec() - ros_t0)*1000 + lps_t1
                    #print(msg.sys_time, lps_time)
                    msg.sys_time = int(lps_time)
            outbag.write(topic, msg, msg.header.stamp if msg._has_header else t)